In [34]:
df['PARAM_VALUE'] = df['PARAM_VALUE'].apply(lambda x: round(pd.to_numeric(x, downcast='float'),6))
df = df.reindex(columns = df.columns.to_list(),  fill_value = np.NaN).where((pd.notnull(df)), None)
df[df.PARAM_VALUE.isna()]

,KPI,OPERATOR,AREA_CAT,AREA,CAT_1,CAT_2,CAT_3,PERIOD_START,PERIOD_END,PARAM_VALUE
319181,NPS,МТС,Region new,Магадан,Area,Central city,None,2020-04-01,2020-04-03,None
319182,NPS,МТС,Region new,Магадан,Area,Others,None,2020-04-01,2020-04-03,None
319183,NPS,МТС,Region new,Магадан,ARPU declared,Low,None,2020-04-01,2020-04-03,None
319184,NPS,МТС,Region new,Магадан,ARPU declared,Middle,None,2020-04-01,2020-04-03,None
319185,NPS,МТС,Region new,Магадан,ARPU declared,High,None,2020-04-01,2020-04-03,None
...,...,...,...,...,...,...,...,...,...,...
363296,"Удобный, простой",МТС,Region,Магадан,None,None,None,2020-10-01,2020-10-03,None
363297,"Удобный, простой",Билайн,Region,Магадан,None,None,None,2020-10-01,2020-10-03,None
363298,"Удобный, простой",Мегафон,Region,Магадан,None,None,None,2020-10-01,2020-10-03,None
363299,"Удобный, простой",Tele2,Region,Магадан,None,None,None,2020-10-01,2020-10-03,None


In [59]:
with teradatasql.connect() as con:
    with con.cursor() as cur:
 
        print('deleting from uat_product.TRACKING_PARAMETERS...')
        cur.execute('''
            delete from uat_product.TRACKING_PARAMETERS;
        ''')
        print(f'{cur.rowcount} rows deleted from uat_product.TRACKING_PARAMETERS')
        
        batchsize = 100000
        print('inserting into uat_product.TRACKING_PARAMETERS...')
        for num in range(0, len(df), batchsize):
            cur.executemany(f'''
             INSERT into uat_product.TRACKING_PARAMETERS ({','.join('?'*len(df.columns))})
            ''',
                [tuple(row) for row in df.iloc[num:num+batchsize,:].itertuples(index=False)]
                )
            print(f'{batchsize} rows inserted...')
        print(f'{len(df)} rows total inserted into uat_product.TRACKING_PARAMETERS.')
        
        print('collecting statistics...')
        cur.execute('''
        COLLECT STATISTICS
            COLUMN(KPI)
            ,COLUMN(OPERATOR)
            ,COLUMN(AREA_CAT)
            ,COLUMN(AREA)
            ,COLUMN(CAT_1)
            ,COLUMN(CAT_2)
            ,COLUMN(CAT_3)
            ,COLUMN(PERIOD_START)
            ,COLUMN(PERIOD_END)
            ,COLUMN(PARAM_VALUE)
            ON UAT_PRODUCT.TRACKING_PARAMETERS;
                ''')
        print(f'{cur.rowcount} columns done.')

deleting from uat_product.TRACKING_PARAMETERS...
0 rows deleted from uat_product.TRACKING_PARAMETERS
inserting into uat_product.TRACKING_PARAMETERS...
100000 rows inserted.
100000 rows inserted.
100000 rows inserted.
100000 rows inserted.
363316 rows total inserted into uat_product.TRACKING_PARAMETERS.
collecting statistics...
11 columns done.


In [53]:
mcrregions_dict = {
'Center':'Центр',
'Chernozem':'Черноземье',
'Baikal & Far East':'Байкал и Дальний Восток',
'North-West':'Северо-Запад',
'Siberia':'Сибирь',
'South':'Юг',
'Volga':'Волга',
'Ural':'Урал',
'Moscow':'Москва и область',
}

df[df.AREA == 'Moscow'].AREA = df.AREA.replace(mcrregions_dict, inplace = True)